In the name of God  

# RWP

In [7]:
#Import library 
from pymobility.models.mobility import random_waypoint

In [25]:
rw = random_waypoint(6, dimensions=(500, 500), velocity=(1.0, 1.0), wt_max=1.0)

In [9]:
positions = next(rw)
positions

d:\Omid\Myfiles\University\پروژه‌ی ارشد\python\testproject\pymobility\models\mobility.py:30: RuntimeWarning: invalid value encountered in double_scalars
  alpha1 = ((pause_high+pause_low)*(speed_high-speed_low))/(2*np.log(speed_high/speed_low))


array([[4.8134366 , 1.52991834],
       [3.29779468, 4.60069091],
       [3.95780063, 4.3794012 ],
       [1.02173898, 3.00692807],
       [0.91809198, 2.84981861],
       [3.09117438, 1.54719884]])

In [ ]:
for positions in rw:
    print(positions)

In [31]:
from tkinter import *
from time import time, sleep
import math

canvas_width = 500
canvas_height = 500
scale = 500 / 500

# dimension of users
rectangle_width = 5
rectangle_height = 5


Nu = 6 #Number of users

# def paint(event):
#     python_green = "#476042"
#     x1, y1 = (event.x - 1), (event.y - 1)
#     x2, y2 = (event.x + 1), (event.y + 1)
#     w.create_oval(x1, y1, x2, y2, fill=python_green)

def showMovement():
        global position
        for i in range(Nu):
                x1 = position[i][0]
                y1 = position[i][1]
                position = next(rw)
                x2 = position[i][0]
                y2 = position[i][1]
                dx = x2 - x1
                dy = y2 - y1
                w.move(user1,dx*scale,dy*scale)
                p
        w.after(50,showMovement)


master = Tk()
master.title("Room")
w = Canvas(master,
           width=canvas_width,
           height=canvas_height,background="white")
w.pack(expand=YES, fill=BOTH)

position = next(rw)
users=[]
for n in range(Nu):
    users[n] = w.create_rectangle(position[n][0]*scale,position[n][1]*scale,
                    position[n][0]*scale+rectangle_width,
                    position[n][1]*scale+rectangle_height,fill='blue')

w.after(50, showMovement)
mainloop()    